<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 2: Pretraining and Supervised Finetuning</h2>
<h4>Lecture: Prof. Michalis Vazirgiannis<br>
Lab: Dr. Hadi Abdine and Yang Zhang</h4>
<h5>Tuesday, October 07, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 12
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>


## <b>Instruction Finetuning</b>



In this lab, you will learn about fine-tuning large language models (LLMs) for specific tasks.

Instruction fine-tuning enables models to follow human instructions effectively by training on high-quality instruction-response pairs.

We will implement these techniques using Python and the Hugging Face ecosystem, including transformers and datasets,  By the end of the lab, you will have hands-on experience in adapting LLMs to specific use cases and evaluating their performance.

In summary, we will:

* Finetune [Qwen2-0.5B](https://huggingface.co/Qwen/Qwen2-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instruction tuning.

<center>
<img src='https://onedrive.live.com/embed?resid=AE69638675180117%21292802&authkey=%21AO_qaECmI1InIyg&width=634&height=556' width="500">


LoRA: Low Rank Adapataion. Taken from LoRA original paper

<img src='https://onedrive.live.com/embed?resid=AE69638675180117%21292801&authkey=%21AIBM2HNKRF7tzGo&width=1980&height=866' width="700">

QLoRA. Taken from QLoRA original paper

</center>

### <b>Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>


In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) - a powerful open-weight family of language models known for a  strong multilingual and reasoning capabilities - on a question answering dataset.

Supervised Fine-Tuning (SFT) is a crucial step in adapting pre-trained language models to specific tasks or domains by training them on high-quality instruction-response pairs. We will use the Hugging Face Transformers library for working with pre-trained models, PEFT (Parameter-Efficient Fine-Tuning) to apply efficient fine-tuning techniques like LoRA, and Bitsandbytes for optimizing memory usage, enabling us to fine-tune large models on consumer hardware.

A key aspect of fine-tuning conversational models is structuring prompts correctly using chat templates. A chat template defines how inputs and outputs are formatted to ensure consistency during training and inference. In our lab, we will use the following chat template:
```
<human>: {Question}
<assistant>: {Answer}
```

Such formats helps the model differentiate between user inputs and assistant responses, ensuring better alignment with real-world chat applications.

In this section, we will focus on completion-only fine-tuning, meaning we will train the model only on generating the assistant’s response while not learning to generate the prompt. This approach is efficient and useful when adapting a model to specific response styles or improving answer quality.



#### <b>Preparing the environment and installing libraries:<b>

In [ ]:
!nvidia-smi

Sun Oct 12 19:27:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 24.9 MB/s eta 0:00:00


In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModelForCausalLM
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

#### <b>Loading the model and the tokenizer:</b>



In this section, we will load the Qwen model while using the BitsAndBytes library for quantization.

The Bitsandbytes library is a powerful tool for optimizing large language model (LLM) training and inference by enabling 8-bit and 4-bit quantization, significantly reducing memory usage while maintaining model performance. Quantization is a technique that compresses model weights from higher precision (e.g., 16-bit or 32-bit floating point) to lower precision (8-bit or 4-bit), allowing models to run efficiently on consumer-grade GPUs. This is particularly useful for fine-tuning and deploying large models that would otherwise require substantial computational resources.

In Bitsandbytes, key parameters control how quantization is applied:

- **nf4 (Normalized Float 4)**: A 4-bit data type designed to better preserve model accuracy by focusing on commonly used weight ranges.
- **bnb_4bit_compute_dtype**.
- **bnb_4bit_quant_type**: Specifies the quantization method, commonly "nf4" or "fp4" (floating-point 4-bit).
- **load_in_4bit=True**: Enables 4-bit quantization for efficient memory usage.
- **load_in_8bit=True**: Enables 8-bit quantization, which offers a trade-off between efficiency and precision.
- **bnb_4bit_use_double_quant**.

Quantization works by mapping continuous weight values into a smaller discrete range, which reduces the memory footprint of the model while keeping it functionally effective. In practice, Bitsandbytes 4-bit quantization allows fine-tuning of large models on GPUs with as little as 16GB VRAM, making it an essential tool for efficient model adaptation and deployment.

In our lab, we will store the model in the VRAM with 4 bits using the 'nf4' quantization method, do the computation using brain float 16 (BF16) and use double quantization.

In [1]:
model

NameError: name 'model' is not defined

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # to go further, try llama with unsloth

bnb_config = BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=torch.bfloat16,
              bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

#### <b>Configuring LoRA:</b>

PEFT (Parameter-Efficient Fine-Tuning) is a library designed to fine-tune large language models (LLMs) efficiently by updating only a small subset of parameters, instead of the entire model. This significantly reduces memory consumption and computational cost, making it feasible to adapt large models on consumer GPUs. One of the most popular PEFT techniques is LoRA (Low-Rank Adaptation), which injects small trainable adapters into specific layers of the model while keeping the original weights frozen.

Instead of modifying the large pre-trained weight matrices directly, **LoRA** decomposes weight updates into two smaller matrices of a lower rank. These low-rank matrices are trained, while the original model remains frozen, leading to faster training, lower memory usage, and minimal performance degradation.

When applying LoRA using PEFT, several important parameters are used:

- **r (Rank)**: The rank of the low-rank matrices added to the model.
Common Practice: Values like 8, 16, or 32 are often used. Higher ranks improve model adaptability but require more memory. In our lab we will use a LoRA rank of 32.
- **lora_alpha**: The scaling factor for LoRA updates.
Common Practice: Set as 2 × rank (e.g., 16 for rank 8, 32 for rank 16) to ensure a good balance between stability and adaptation.
- **lora_dropout**: Dropout applied to LoRA layers to prevent overfitting.
Common Practice: 0.05–0.1 is commonly used. In our lb we will use 0.05.
- **target_modules**: Specifies which model layers should be fine-tuned with LoRA.
Common Practice: For transformer models like LLaMA, Qwen, and Mistral, LoRA is typically applied on all projection (MLP) layers inside the transformer block (so excluding the embedding and language modeling head layers).

 **Note:** set `bias` to `'none'` and do not forget to set the `task_type` to the causla language modeling task.

In [ ]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e

In [ ]:
config = LoraConfig(
    task_type="CAUSAL_LM",
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias='none'
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2162688 || all params: 317282176 || trainable%: 0.6816292132338376


#### <b>Test the model before finetuning:</b>

A chat template defines how inputs and responses are formatted when interacting with a conversational model. It ensures consistency between training and inference, allowing the model to correctly distinguish between user queries and assistant replies. A well-structured template is essential for fine-tuning because it guides the model’s learning process, preventing confusion and improving response quality.

As mentioned before, in this lab, we will use the following chat template:

```
<human>: {Question}
<assistant>: {Answer}
```

In [ ]:
prompt = "<human>: {What equipment do I need for rock climbing}"
print(prompt)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.do_sample = True

<human>: {What equipment do I need for rock climbing}


In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<human>: {What equipment do I need for rock climbing} 1. Rock climbing harness
2. Rock climbing gloves
3. Rock climbing helmet
4. Rock climbing board
5. Rock climbing shoes
6. Rock climbing backpack
7. Rock climbing pack
8. Rock climbing safety equipment (belt, rope, etc.)
9. Rock climbing gear (cable, rope, etc.)
10. Rock climbing gear (helmets, gloves, etc.) 1. Rock climbing harness
2. Rock climbing gloves
3. Rock climbing helmet
4. Rock climbing board
5. Rock climbing shoes
6. Rock climbing backpack
7. Rock climbing pack
8. Rock climbing safety equipment (belt, rope, etc.)
9. Rock climbing gear (cable, rope, etc.)
10. Rock climbing gear (helmets, gloves, etc.)
CPU times: user 18.4 s, sys: 291 ms, total: 18.7 s
Wall time: 27.4 s


In [ ]:
# encoding.input_ids
# encoding.attention_mask

#### <b>Loading the question answering dataset from Hugging Face Hub:</b>

For fine-tuning our model, we will use the `giuliadc/orangesum_5k` dataset, a high-quality collection of articles-summaries pairs. This dataset contains news articles written in French.

Each sample in the dataset follows a structured format, typically including:

- **id:** The id of the article
- **text:** The original text of the article.
- **reference-summary:** The summary of the article.

In [ ]:
data = load_dataset("giuliadc/orangesum_5k")
data_pd = pd.DataFrame(data["train"])
data_pd

README.md: 0.00B [00:00, ?B/s]

orangesum_ids_train.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

,id,text,reference-summary
0,orangesum-1,Emmanuel Macron s'est montré défavorable à une...,Le président aurait sèchement écarté l'idée de...
1,orangesum-2,Elle a été interpellée mardi 16 juin sans ména...,Elle a été filmée lançant des projectiles sur ...
2,orangesum-3,La confiance des Français à l'égard des financ...,SONDAGE. Quarante six pour cent des personnes ...
3,orangesum-4,"""L'affaire dure. (...) Mais cette histoire ne ...",C'est un soutien de poids. L'ancienne ministre...
4,orangesum-5,"""On n'a rien demandé! On est des gens honnêtes...","A Moissac, l'heure est à la cueillette des pre..."
...,...,...,...
4995,orangesum-4996,"A Vacaville, une ville d'environ 100.000 habit...",Des milliers de personnes ont fui leurs maison...
4996,orangesum-4997,"Une semaine après Dieudonné, c'est au tour de ...",Ce proche de Dieudonné ne pourra pas recréer d...
4997,orangesum-4998,Au moins dix personnes sont mortes et une autr...,Après l'incendie qui a fait 10 morts dans la n...
4998,orangesum-4999,"""Si l'on parvient à observer une étoile qui se...","La mission d'astronomie sino-française Svom, v..."


In [ ]:
# data_pd.iloc[0]['text']
data["train"].shuffle(seed=42)['text']

Column(['"Ce couple de condors est le plus impressionnant et le plus prolifique que nous connaissons pour cette espèce" de Vultur gryphus, explique à l\'AFP le biologiste Sebastian Kohn, directeur de la Fondation Condors andins qui collabore avec le ministère de l\'Environnement. L\'espèce, connue communément sous le nom de condor des Andes, est présente en Amérique du sud, tout au long de la Cordillère des Andes. Son envergure de 3,5 mètres et son poids d\'environ 15 kg en font un des plus grands oiseaux du monde. Le couple observé par les chercheurs a son domaine autour du volcan Antisana, à 50 km au sud-est de Quito, où ils ont installé leur nid sur un piton rocheux, dans la réserve naturelle de Chakana. "Depuis 2013 que nous les étudions, ils ont déjà eu sept petits", raconte M. Kohn, qui avec son équipe les observe avec des jumelles et des appareils photographiques depuis un mirador de la réserve. Or un couple de condors des Andes, une espèce monogame, a en général un petit tous l

#### <b>Preparing the finetuning data:</b>

Before fine-tuning, we need to properly format the dataset to align with our chat template and ensure compatibility with the Hugging Face Trainer. Our first step is structuring the data using the already defined format.

Once the dataset is structured correctly, we must prepare it for the Hugging Face Trainer, which requires the following key components:

- **`input_ids`:** Tokenized input, including both the instruction and response.
- **`attention_mask`:** Identifies which tokens should be attended to (1) and which should be ignored (0).
- **`labels`:** Defines the target output during training.

Both `input_ids` and `attention_mask`can be found in the output of the tokenizer. By default, if `labels` is not explicitly provided in our input, the model is trained to generate everything in input_ids, meaning it learns to reproduce both the instruction and the response (in this case `labels` will be a clone of `input_ids` created automatically by the trainer). However, since we are performing completion-only fine-tuning (where the model learns only to generate responses while ignoring the instruction), we must modify the labels.

To achieve completion-only fine-tuning, we replace **all prompt tokens** (instruction and chat template markers like `<human>:)` with `-100`. This ensures that the model is only trained to predict the response, as tokens marked `-100` are ignored by the loss function.

In [ ]:
def generate_prompt(data_point):
    article = data_point['text']
    summary = data_point['reference-summary']
    texto = f"<human>: Résumez l’article suivant:\n{article}?\n <assistant>: {summary}"
    return texto

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)+tokenizer.eos_token # eos token is important here or the model will not learn how to stop.
    tokenized_full_prompt = tokenizer(full_prompt, return_tensors='pt')
    if tokenized_full_prompt.input_ids.shape[1] > 2000:
        return None

    labels = tokenized_full_prompt.input_ids.clone()

    prompt = full_prompt[:full_prompt.find("<assistant>")] + "<assistant>:"

    end_prompt_idx = len(tokenizer(prompt).input_ids)

    labels[:, :end_prompt_idx] = -100

    return {
        'input_ids': tokenized_full_prompt.input_ids.flatten(),
        'labels': labels.flatten(),
        'attention_mask': tokenized_full_prompt.attention_mask.flatten(),
    }

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
print(data['input_ids'][1])
print(data['labels'][1])

[9969, 7136, 26818, 50123, 1242, 10125, 326, 527, 7058, 45832, 517, 510, 7044, 75, 19694, 70906, 409, 71, 1087, 11, 389, 822, 275, 1709, 272, 17321, 24901, 4581, 75266, 1842, 1187, 15632, 367, 1788, 63958, 41038, 939, 5234, 32803, 409, 65578, 497, 264, 312, 5148, 84, 77018, 647, 963, 6576, 11, 7774, 733, 459, 275, 45839, 4704, 837, 8579, 6185, 330, 4475, 28882, 2165, 1, 939, 330, 1580, 886, 3723, 1, 3784, 1187, 5208, 4517, 3845, 330, 79, 544, 1, 409, 326, 6, 40103, 2645, 13, 1967, 48804, 265, 409, 1187, 22896, 963, 264, 13527, 622, 963, 922, 34229, 42889, 330, 83, 887, 259, 34557, 4914, 44612, 261, 939, 30242, 1, 19812, 17276, 409, 1187, 22638, 7888, 78888, 6712, 3845, 65578, 409, 1187, 7042, 11, 512, 220, 16, 16, 16737, 11, 26486, 274, 34229, 308, 58758, 264, 6368, 4914, 326, 6, 74776, 409, 1841, 288, 294, 30009, 330, 265, 1786, 299, 23965, 1, 409, 1187, 34755, 3845, 32233, 13, 25968, 3541, 4445, 3541, 13216, 1723, 14789, 42993, 95208, 78613, 549, 512, 20316, 276, 409, 326, 30669, 104

In [ ]:
# data["train"].shuffle(seed=42)

#### <b>Finetuning:</b>

Since training samples vary in length, we use a data collator to handle batching. Specifically, we use `DataCollatorForSeq2Seq`, which:

- Pads inputs and attention masks to the longest sequence in the batch.
- Ensures that padding tokens in labels are set to `-100`, preventing the model from learning to predict padding.

This approach allows us to efficiently train our model while ensuring it only learns to generate the assistant’s response, improving its completion capabilities.

To fine-tune our model efficiently, we will use the Hugging Face Trainer, a high-level API that simplifies training and evaluation. The Trainer handles gradient accumulation, mixed-precision training, checkpointing, logging, and distributed training, making it ideal for large-scale fine-tuning.

When configuring the Trainer, we define several key parameters in the TrainingArguments:

- `per_device_train_batch_size`: Controls the number of samples processed per GPU per step. Smaller values (e.g., 2, 4) are used for memory efficiency.
- `gradient_accumulation_steps`
- `num_train_epochs`: Defines how many times the model sees the entire dataset during training (typically 2–3 epochs for fine-tuning).
- `learning_rate`: Determines how much the model adjusts weights per step. A low learning rate (e.g., 2e-5) helps prevent catastrophic forgetting.
- `lr_scheduler_type`: Controls how the learning rate decays over time (e.g., "cosine" or "linear" are commonly used).
- `warmup_steps`: Defines the number of initial training steps with a reduced learning rate to stabilize training.
- `logging_steps`: Specifies how often training metrics (e.g., loss) are logged.
save_steps: Determines how frequently model checkpoints are saved.
- `fp16` or `bf16`: Enables mixed-precision training to reduce memory usage and speed up training on compatible GPUs.
- `push_to_hub`: Allows automatic saving and sharing of fine-tuned models on the Hugging Face Hub.

Once the Trainer is set up, training starts with the `.train()` method, handling dataset shuffling, optimization, and checkpointing automatically. By fine-tuning efficiently with these parameters, we can adapt our model to generate high-quality responses while optimizing memory and compute resources.

P.S. it is normal if you do not see loss decrease in this PoC. (Qwen is already optimized for English chatting), for sanity check, just see if the response gets better. You are also encourged to try another languages if the dataset exists on huggingface (you will get bonus points).

In [ ]:
from transformers import DataCollatorForSeq2Seq

OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    learning_rate=5e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir=OUTPUT_DIR,
    max_steps=100,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.01,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

model.config.use_cache = False
trainer.train()

Step,Training Loss
20,1.950000
40,1.853700
60,1.883600
80,1.839600
100,1.842700


TrainOutput(global_step=100, training_loss=1.8739009094238281, metrics={'train_runtime': 1605.8155, 'train_samples_per_second': 0.996, 'train_steps_per_second': 0.062, 'total_flos': 2863369109568000.0, 'train_loss': 1.8739009094238281, 'epoch': 0.3214788024914607})

In [ ]:
# %load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

UsageError: Line magic function `%tensorboard` not found.


#### <b>Test the model after the finetuning (out-of-distribution prompt):<b>

In [ ]:
%%time
device = "cuda:0"
## uncomment if you didn't have enough time to train
# model = AutoModelForCausalLM.from_pretrained(
#                     MODEL_NAME,
#                     device_map="auto",
#                     trust_remote_code=True,
#                     quantization_config=bnb_config,
#                 )
# model = PeftModelForCausalLM.from_pretrained(model, "habdine/CSC_53432_lab2")

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: {What equipment do I need for rock climbing} - A rock climber needs a set of equipment to perform their sport. The equipment includes: helmet, gloves, backpack, rope, climbing harness, climbing shoes, and other climbing accessories.
CPU times: user 2.89 s, sys: 799 µs, total: 2.89 s
Wall time: 2.9 s


In [ ]:
def generate_response(prompt: str) -> str:
    prompt = f"<human>: {prompt}\n<assistant>:"
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start:]

In [ ]:
prompt = f""""Ce couple de condors est le plus impressionnant et le plus prolifique que nous connaissons pour cette espèce" de Vultur gryphus, explique à l\'AFP le biologiste Sebastian Kohn, directeur de la Fondation Condors andins qui collabore avec le ministère de l\'Environnement. L\'espèce, connue communément sous le nom de condor des Andes, est présente en Amérique du sud, tout au long de la Cordillère des Andes. Son envergure de 3,5 mètres et son poids d\'environ 15 kg en font un des plus grands oiseaux du monde. Le couple observé par les chercheurs a son domaine autour du volcan Antisana, à 50 km au sud-est de Quito, où ils ont installé leur nid sur un piton rocheux, dans la réserve naturelle de Chakana. "Depuis 2013 que nous les étudions, ils ont déjà eu sept petits", raconte M. Kohn, qui avec son équipe les observe avec des jumelles et des appareils photographiques depuis un mirador de la réserve. Or un couple de condors des Andes, une espèce monogame, a en général un petit tous les deux ou trois ans. Selon le chercheur, la bonne reproduction des prédateurs peut s\'expliquer par un bon accès à de la nourriture, notamment des charognes d\'animaux, et la sensation de sécurité que leur procure la réserve naturelle. - Empoisonnements -Mais cette situation n\'empêche pas le biologiste de s\'inquiéter pour l\'avenir de l\'espèce qui compte 150 individus en Equateur, selon un recensement datant de 2018. Selon lui, le pays devrait relever le niveau d\'alerte à "danger critique". Au niveau mondial, l\'espèce, qui compte 6.700 individus, est considérée par l\'Union internationale de la conservation de la nature (UICN), comme "presque menacée", avec des effectifs en baisse régulière. Sur un autre piton rocheux de la réserve, batipsé le piton du Condor, à environ 4.100 mètres d\'altitude, se trouve le principal perchoir où une quarantaine d\'oiseaux ont pu être aperçus. Au cours des deux dernières années, "nous avons perdu 15 à 20 individus, principalement à cause d\'empoisonnements" liés à la consommation de charognes contaminées destinées aux prédateurs de bétail, "mais aussi à cause de la chasse", souligne M. Kohn. En septembre, Iguiñaro, un condor relâché en mai dans la réserve de Chakana, a été retrouvé mort après avoir été soigné de ses blessures provoquées par des tirs de chasseurs. "Il y a un siècle, on pouvait voir jusqu\'à 100 condors. Aujourd\'hui, si vous avez de la chance, vous en voyez dix", se désole le chercheur.', '"Il fait beau dehors, on sait que c\'est très compliqué et la tentation est forte après des semaines de confinement", a reconnu Olivier Véran, qui constatait déjà mercredi une "accélération" des "regroupements" à la veille du "pont" de l\'Ascension. Le ministre de la Santé a souligné qu\'il était "trop tôt pour tirer des conclusions" sanitaires de la levée partielle du confinement de la population, le 11 mai, même s\'il n\'y a pas pour l\'heure de signes d\'une "re-croissance" de la circulation du coronavirus. Pour les moment les indicateurs sont toujours plutôt meilleurs : le bilan de l\'épidémie de coronavirus s\'établit jeudi à au moins 28.215 morts, avec 83 nouveaux décès enregistrés depuis la veille, selon la Direction générale de la santé. Le nombre de patients en réanimation, indicateur important de la pression sur le système hospitalier,a continué à décroître, avec 1.745 cas graves en réanimation, soit 49 de moins en 24 heures. Pour la première fois depuis le plus fort de la crise, la Nouvelle Aquitaine - l\'une des régions toutefois les moins touchées - ne compte aucun mort, a souligné l\'Agence régionale de santé. Si les contrôles routiers étaient nombreux pour faire respecter la règle de déplacements limités à 100 kilomètres autour du domicile, beaucoup avaient visiblement hâte de partir: la circulation était ainsi dense jeudi matin sur l\'autoroute A6 au départ de Paris, avant de se calmer dans l\'après-midi. Cinq communes du Morbihan et une du Finistère ont demandé la fermeture de leurs plages, rouvertes le week-end dernier, après avoir constaté le non respect des règles de distanciation. La mairie de Saint-Malo (Ille-et-Vilaine) a rendu le port du masque obligatoire de 11H00 à 18H00 dans certaines rues très commerçantes. Des policiers municipaux patrouillaient pour s\'en assurer, et un crieur de rue déguisé en corsaire rappelait les règles de distanciation.- Brutal et intense -Ailleurs, plages et plans d\'eau ont continué à rouvrir, notamment en Méditerranée, le plus souvent en "mode dynamique" : pas question de s\'asseoir ou de faire des châteaux de sable, encore moins de s\'allonger pour bronzer. En Occitanie, des dizaines de lacs et plans d\'eau, notamment dans le massif pyrénéen, ont été rouverts au public pour la promenade, la pêche et quelques activités nautiques. Baignade et pique-nique restent interdits, comme les groupes de plus de 10 personnes. Il existe toutefois des exceptions: à la Grande-Motte, il est possible de bronzer, à condition d\'avoir réservé à l\'avance son emplacement, délimité par des cordes. Idem à Leucate, où l\'on peut s\'étendre sur la plage pendant deux heures, avec une distance de cinq mètres entre chaque groupe. "On souhaitait expérimenter un système permettant le bain de soleil en toute sécurité, sur 400 mètres de plage", explique le maire Michel Py. En Martinique, les plages de 12 des 24 communes du littoral ont également rouvert au public jeudi, après celles de plusieurs municipalités de Guadeloupe la veille. Le parc d\'attraction vendéen Le Puy du Fou a annoncé de son côté une réouverture le 11 juin, affirmant avoir été informé d\'une décision du président de la République de rouvrir les sites touristiques et parcs à thème en zone verte le 2 juin. Le secrétaire d\'Etat au Tourisme Jean-Baptiste Lemoyne a dit se réjouir "que ce travail de préparation à la réouverture, avec comme objectif le 2 juin, soit désormais enclenché avec le Puy du Fou comme avec d\'autres sites de loisirs", dans une déclaration transmise à l\'AFP. Le gouvernement a promis 18 milliards d\'euros pour aider le secteur du tourisme, fortement impacté par le confinement et la crise sanitaire. Le secteur des transports devrait également y laisser des plumes: le constructeur automobile Renault doit dévoiler le 29 mai les contours d\'un vaste plan d\'économies de deux milliards d\'euros. La firme au losange "joue sa survie" a déclaré jeudi soir au Figaro le ministre de l\'Economie Bruno Le Maire, qui attend "des engagements" de la part du groupe sur le maintien en France de certaines activités. Il a notamment souligné qu\'il n\'avait "pas encore" donné son feu vert à un prêt de 5 milliards d\'euros au constructeur en détresse. Le Premier ministre Edouard Philippe a assuré que le gouvernement présenterait la semaine prochaine un plan pour le secteur automobile. Et qu\'il serait "intransigeant sur la préservation des sites (en) France". Bruno Le Maire a également déclaré que la France souhaite que les règles de discipline budgétaire entre membres de l\'Union européenne, suspendues pour 2020 face à la crise du Covid-19, le soient aussi l\'année prochaine. Autre dossier chaud, les municipales, sur lesquelles Edouard Philippe devrait envoyer son rapport au Parlement entre vendredi et samedi, après un avis très prudent de son Conseil scientifique. Une possibilité serait d\'organiser le second tour en juin, très probablement le 28, l\'autre de le repousser jusqu\'à la date butoir de janvier 2021.- Lacunes -Un autre secteur dont la crise a été mise en évidence par l\'épidémie est le système de santé français. La Commission européenne a pointé les "lacunes" de la France dans la préparation à ce type d\'événement. Plusieurs centaines de personnes ont manifesté au son des casseroles jeudi devant l\'hôpital Robert Debré à Paris pour réclamer des "lits" et "du fric pour l\'hôpital public". Des soignants brandissaient des pancartes "blouses blanches colère noire" ou "managers technocrates, hors hôpital", entourés de "gilets jaunes" et d\'élus comme le député LFI Eric Coquerel. Olivier Véran, qui doit lancer lundi une grande consultation des acteurs du secteur, a promis "des mesures d\'ampleur" et "dans une certaine mesure radicales", affichant l\'ambition d\'aboutir mi-juillet. Au menu: des hausses de salaires pour les soignants mais aussi des réorganisations et un allègement du "carcan qui empêche ceux qui le souhaitent de travailler davantage" -- en clair un assouplissement des 35 heures.', 'A la suite des mesures annoncées par le Premier ministre hier soir, le ministre de l\'Éducation a tenu à justifier leur ampleur en rappelant que "depuis le début, la stratégie ce n\'est pas d\'empêcher que le virus passe - on sait qu\'il passera probablement par plus de la moitié d\'entre nous - mais c\'est de faire en sorte qu\'il passe de la manière la plus étalée possible dans le temps." "Le monde navigue à vue [...] nous pilotons au plus près en tenant compte de ce que disent les scientifiques", déclare le ministre de l\'Éducation Jean-Michel Blanquerpar franceinfoLe ministre faisait référence aux personnes potentiellement contaminées mais pas celles - beaucoup moins nombreuses - chez qui le virus prend des formes graves. Continuer le travail"On considère, et là je ne fais que répéter ce que disent les scientifiques, que 50 à 70% de la population in fine finit par être contaminée par le virus, et c\'est d\'ailleurs ça qui met fin au virus puisque ça crée une forme d\'immunité majoritaire, et donc le virus s\'éteint de lui-même", a ajouté le ministre. Le gouvernement a notamment annoncé la fermeture de tous les lieux publics non-indispensables afin de freiner l\'épidémie qui a contaminé 4.500 personnes en France et fait 91 morts, selon le dernier bilan. A partir de lundi, les 850 000 enseignants de France vont devoir assurer les cours à distance, de la grande section de maternelle au lycée, une tâche "surmontable" pour beaucoup mais qui demandera, selon eux, "pas mal de travail et d\'organisation". L\'Education nationale a annoncé mettre à disposition les ressources du Centre national d\'enseignement à distance (Cned): des exercices en ligne adaptés aux programmes et une "classe virtuelle" où le professeur peut faire cours à ses élèves par visioconférence. Le ministre a quant à lui appeler les élèves à continuer le travail en dépit des circonstances.', 'Des manifestants ont bloqué une autoroute et ont incendié le restaurant fast food Wendy\'s près duquel Rayshard Brooks a été abattu lors d\'une confrontation avec la police, ont rapporté les médias locaux. La maire d\'Atlanta, Keisha Lance Bottoms, dont le nom a été avancé comme une possible colistière du démocrate Joe Biden pour l\'élection présidentielle de novembre, a annoncé la démission d\'Erika Shields, qui dirigeait la police d\'Atlanta depuis plus de 20 ans."En raison de son désir qu\'Atlanta soit un modèle de ce qu\'une réforme significative devrait être dans tout le pays, Erika Shields a présenté sa démission immédiate de cheffe de la police", a dit la maire dans des déclarations télévisées. Les faits se sont produits vendredi soir à Atlanta, capitale de l\'Etat de Géorgie. Ils surviennent alors que de nombreuses manifestations contre les violences policières et contre le racisme ont lieu depuis des semaines aux Etats-Unis et dans d\'autres pays, à la suite de la mort de George Floyd, un Afro-Américain mort asphyxié par un policier à Minneapolis, dans le nord des Etats-Unis. Selon un rapport officiel, Rayshard Brooks, un homme noir âgé de 27 ans, s\'était endormi dans sa voiture sur l\'allée du drive-in du restaurant Wendy\'s, et des employés de l\'établissement ont appelé la police parce que son véhicule bloquait les clients. L\'homme était alcoolisé et a résisté lorsque la police a voulu l\'arrêter, indique le rapport du Georgia Bureau of Investigation. Les images de surveillance vidéo montrent "qu\'au cours d\'une lutte physique avec les agents, Brooks s\'est emparé du Taser de l\'un des agents et a pris la fuite", selon le rapport."Les agents ont poursuivi Brooks à pied et pendant la poursuite Brooks s\'est retourné et a pointé le Taser vers l\'agent. L\'agent a utilisé son arme, touchant Brooks", indique le document. Brooks a été transporté vers un hôpital et a été opéré mais il est décédé peu après, déclare le rapport, qui ajoute qu\'un agent a été blessé. La maire a déclaré que l\'agent qui avait procédé au tir mortel devrait être destitué. Ce dernier, identifié comme Garrett Rolfe, a été renvoyé de la police d\'Atlanta, et un deuxième policier a été suspendu, selon la chaîne ABC News.', 'Dernier en date : Jean-François Copé, révèle Le Parisien. Une personnalité de plus s\'ajoute à la longue liste de candidats potentiels à la mairie de Paris. Selon les informations du Parisien, Jean-François Copé lorgnerait lui aussi sur la mairie de Paris. Depuis son gros revers à la primaire de la droite en novembre 2016 (0,3%), le maire de Meaux a quasiment disparu de la scène médiatique. Une lumière qui semble lui manquer. Selon le quotidien, Jean-François Copé aurait désormais les Municipales 2020 en ligne de mire et aurait déjà commencé à tâter le terrain auprès des parlementaires Les Républicains. Malheureusement pour lui, ces derniers lui auraient vite remis les idées en place. "Il est cramé à vie. Déjà que la droite a très peu de chances aux prochaines municipales à Paris. Alors en plus si c\'est lui...", a d\'ailleurs lancé au Parisien l\'un d\'eux. Une course mal engagée donc pour Jean-François Copé, d\'autant que les candidats à la succession d\'Anne Hidalgo ne manquent pas. Le porte-parole du gouvernement, Benjamin Griveaux, en premier lieu. 20 Minutes avançait d\'ailleurs en juin dernier qu\'un groupe de Marcheurs travaillait déjà sur le futur programme.À droite, Rachida Dati s\'était elle aussi dite prête "à mener le combat". " Imaginons que ma famille politique dise \'pourquoi pas ?\', je mènerais le combat, je suis jusqu\'au-boutiste dans les combats, sinon ce n\'est pas la peine", avait indiqué la maire du 7e arrondissement de la capitale à BFMTV.À gauche, l\'ancien conseiller de François Hollande, Gaspard Gantzer semble aussi ouvrir la voie à une candidature, avec le lancement d\'un mouvement baptisé "Parisiennes, Parisiens". Ségolène Royal pourrait elle aussi se laisser tenter. Mais l\'ancienne ministre de l\'Écologie avait confié au Parisien qu\'elle ne se présenterait "que si Anne Hidalgo n\'y allait pas". Et les chances de voir l\'actuelle maire de Paris renoncer restent très faibles.
"""
# print('-', article,'\n\n')
print(generate_response(prompt))



<assistant>: "Les condors ont été observés à la limite de la zone de protection de l'Ascension, à proximité de Chakana, dans le Morbihan, et au niveau de la réserve Chakana, dans le Morbihan, selon un recensement réalisé en 2018.", "Les condors ont été observés à la limite de la zone de protection de l'Ascension, à proximité de Chakana, dans le Morbihan, et au niveau de la réserve Chakana, dans le Morbihan, selon un recensement réalisé en 2018.", "Le gouvernement a annoncé jeudi soir avoir "des mesures d'ampleur" pour aider le secteur du tourisme, notamment pour le système de santé français. Il a également annoncé le plan d'urgence pour le secteur automobile. Les maux de consanguinity de l'Ascension et de la Cordillère des


In [ ]:
prompt = f"""Résumez l’article suivant:
Une petite révolution se prépare. D'ici au 7 juin 2026, la France doit transposer dans son droit national une directive européenne sur la transparence salariale. Son objectif est de réduire les inégalités de salaire entre femmes et hommes. Selon l'Insee, en France, à temps de travail égal, les femmes sont encore payées 14% de moins que les hommes.

'À travail égal, rémunération égale. Et pour parvenir à l’égalité de rémunération, il faut de la transparence. Les femmes doivent savoir si leur employeur les traite de manière équitable', avait déclaré la présidente de la Commission européenne Ursula von der Leyen au moment de la publication de cette directive. Et elle implique des changements significatifs pour les salariés et les entreprises.

Le premier changement concerne la recherche d'emploi. Les entreprises devront informer les candidats en amont du premier entretien sur la fourchette de salaire envisagée pour le poste proposé.

Cela laisse deux options aux employeurs: soit ils affichent une fourchette de salaire directement sur l'offre d'emploi, soit ils la communiquent directement aux candidats qui ont envoyé leur CV avant le premier entretien.

La deuxième obligation est certainement celle qui va le plus bousculer la vie en entreprise. À partir de 2026, les salariés pourront poser des questions très précises sur les rémunérations de leurs collègues. Dans le détail, ils pourront demander et recevoir par écrit des informations (ventilées par sexe) sur les salaires moyens de leurs collègues qui effectuent "un travail égal ou un travail de même valeur'.

Cette disposition 'vise à garantir que les travailleurs puissent se comparer', y compris à des collègues de l'autre sexe, qui ont un poste équivalent. Cela permettra d'aider les salariés à savoir où ils se positionnent. Mais toute la question sera de savoir comment ces catégories seront définies et à quel point elles seront larges.

La directive impose une réponse 'circonstanciée' et l’obligation pour l’employeur si une différence de rémunération est constatée sans être justifiée par des critères objectifs non sexistes de "remédier" à la situation.

Le salarié pourra aussi demander des précisions sur les critères d'évolution salariale. Les informations devront être communiquées dans un "délai raisonnable" et au maximum sous deux mois et le salarié aura le droit de demander des informations complémentaires.
"""
# print('-', article,'\n\n')
print(generate_response(prompt))


# test the model on out-of-distribution prompt 2 :
prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

<assistant>: L'Insee a déclaré que les femmes étaient encore payées 14% moins que les hommes. La directive européenne sur la transparence salariale sera mise en place en France en 2026.



- Do you know the reasons as to why people love coffee so much? 

<assistant>: Do you know the reasons as to why people love coffee so much?


#### **Merging the main model with the adapter**

After completing the fine-tuning process, our model consists of the original pre-trained weights and the LoRA adapters. Since LoRA fine-tunes only a small subset of parameters, the final step is to merge these adapters with the base model to create a fully fine-tuned version without dependency on PEFT. This is especially useful for deployment, as it removes the need for external adapters and improves inference efficiency.

To merge the LoRA weights, we use the `merge_and_unload()` method from PEFT, which integrates the trained LoRA layers into the base model. Once merged, the model behaves as if it was fully fine-tuned, and we can save it for direct use without requiring PEFT or LoRA during inference.

In [ ]:
model # check the model architecture with the added LoRA layers.

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bi

In [ ]:
model = model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
model # check the model architecture after merging.

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e

To go further:
- Check **VLLM** for fast batch inference.
- Check **DDP**, **FSDP** and **Deepspeed** for distributed training with Hugging Face transformers.
- Check **unsloth** for faster training.
- Check **ollama** for chatting interface.
- Test **multi-turn** and **few-shot learning**.
- Check **Megatron, Nanotron, etc..** for distributed **pre-training** on big clusters.
- Check **LLama Factory** (https://github.com/hiyouga/LLaMA-Factory) for **Finetuning**.